<a href="https://colab.research.google.com/github/UrielMendoza/LANOT_USAC_taller_sentinel2/blob/main/ejemplo_API_openeo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notebook 2: Cálculo de EVI y AFAI con openeo**

Autor: urielm

Fecha: 20 de noviembre de 2024

Introducción

Este notebook guía el cálculo de índices como el EVI y el AFAI utilizando datos Sentinel-2 y el cliente Python de openEO.


In [2]:
!pip install openeo rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.4 MB/s eta 0:00:00


In [17]:
import openeo
import rasterio
import matplotlib.pyplot as plt
import numpy as np

In [12]:
# Conectar al backend de openEO
connection = openeo.connect("openeo.dataspace.copernicus.eu")
connection.authenticate_oidc()


Authenticated using refresh token.


<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>

2. Crear el cubo de datos

In [13]:
def create_datacube(spatial_extent, temporal_extent, max_cloud_cover=20):
    """
    Crea un cubo de datos Sentinel-2 L2A para un área y rango de fechas.
    """
    return connection.load_collection(
        "SENTINEL2_L2A",
        spatial_extent=spatial_extent,
        temporal_extent=temporal_extent,
        bands=["B02", "B04", "B08", "B11"],
        max_cloud_cover=max_cloud_cover,
    )


3. Cálculo de EVI

In [14]:
def calculate_evi(datacube):
    """
    Calcula el índice EVI a partir del cubo de datos.
    """
    blue = datacube.band("B02") * 0.0001
    red = datacube.band("B04") * 0.0001
    nir = datacube.band("B08") * 0.0001
    return 2.5 * (nir - red) / (nir + 6 * red - 7.5 * blue + 1)



5. Cálculo y visualización del EVI

Parámetros de interés para EVI

In [15]:
# Coordenadas y fecha para EVI
spatial_extent_evi = {"west": -90.96, "south": 14.40, "east": -90.95, "north": 14.41}
temporal_extent_evi = ["2024-08-21", "2024-08-21"]

# Crear el cubo de datos
datacube_evi = create_datacube(spatial_extent_evi, temporal_extent_evi)

# Calcular EVI
evi_cube = calculate_evi(datacube_evi)

# Descargar el resultado
evi_cube.max_time().download("evi_composite.tif")


Visualización del EVI


In [35]:
# Visualizar el mapa y el histograma del EVI
with rasterio.open("evi_composite.tif") as src:
    evi_data = src.read(1)
    evi_data = np.where(evi_data > 0, evi_data, np.nan)  # Eliminar valores negativos

plt.figure(figsize=(12, 5))

# Mapa de EVI
#plt.subplot(1, 2, 1)
#plt.imshow(evi_data, cmap="YlGn", vmin=0, vmax=1)
#plt.colorbar(label="EVI")
#plt.title("Mapa EVI")

# Histograma de EVI
#plt.subplot(1, 2, 2)
#plt.hist(evi_data.flatten(), bins=50, color="green", edgecolor="black")
#plt.title("Histograma de EVI")
#plt.xlabel("EVI")
#plt.ylabel("Frecuencia")

#plt.tight_layout()
#plt.show()


<Figure size 1200x500 with 0 Axes>

<Figure size 1200x500 with 0 Axes>

6. Función para calcular AFAI


In [19]:
def calculate_afai(datacube):
    """
    Calcula el índice AFAI a partir del cubo de datos.
    """
    red = datacube.band("B04") * 0.0001
    nir = datacube.band("B08") * 0.0001
    swir = datacube.band("B11") * 0.0001
    return nir - (red + (swir - red) * 0.5)


7. Cálculo y visualización del AFAI

Parámetros de interés para AFAI

In [27]:
# Coordenadas y fecha para AFAI
spatial_extent_afai = {"west": -88.25, "south": 16.36, "east": -88.20, "north": 16.39}
temporal_extent_afai = ["2024-05-05", "2024-05-05"]

# Crear el cubo de datos
datacube_afai = create_datacube(spatial_extent_afai, temporal_extent_afai)

# Calcular AFAI
afai_cube = calculate_afai(datacube_afai)

# Descargar el resultado
afai_cube.max_time().download("afai_composite.tif")


Visualización del AFAI

In [36]:
# Visualizar el mapa y el histograma del AFAI
with rasterio.open("afai_composite.tif") as src:
    afai_data = src.read(1)
    afai_data = np.where(afai_data > 0, afai_data, np.nan)  # Eliminar valores negativos

plt.figure(figsize=(12, 5))

# Mapa de AFAI
#plt.subplot(1, 2, 1)
#plt.imshow(afai_data, cmap="cool", vmin=0, vmax=0.5)
#plt.colorbar(label="AFAI")
#plt.title("Mapa AFAI")

# Histograma de AFAI
#plt.subplot(1, 2, 2)
#plt.hist(afai_data.flatten(), bins=50, color="blue", edgecolor="black")
#plt.title("Histograma de AFAI")
#plt.xlabel("AFAI")
#plt.ylabel("Frecuencia")

#plt.tight_layout()
#plt.show()


<Figure size 1200x500 with 0 Axes>

<Figure size 1200x500 with 0 Axes>